### learning rate = 0.005, pos tags predicted using h_final

In [1]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.WARN)
import pickle
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import os
from tensorflow.python.client import device_lib
from collections import Counter
import time

In [2]:
f = open('../../Glove/word_embedding_glove', 'rb')
word_embedding = pickle.load(f)
f.close()

word_embedding = word_embedding[: len(word_embedding)-1]

f = open('../../Glove/vocab_glove', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"

# Model Description
model_name = 'model-aw-lex-2-2'
model_dir = '../output/all-word/' + model_name
save_dir = os.path.join(model_dir, "save/")
log_dir = os.path.join(model_dir, "log")

if not os.path.exists(model_dir):
    os.mkdir(model_dir)
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

with open('/data/aviraj/dataset/train_val_data_fine/all_word_lex','rb') as f:
    train_data, val_data = pickle.load(f)    
    
# Parameters
mode = 'train'
num_senses = 45
num_pos = 12
batch_size = 64
vocab_size = len(vocab)
unk_vocab_size = 1
word_emb_size = len(word_embedding[0])
max_sent_size = 200
hidden_size = 256
num_filter = 256
kernel_size = 5
keep_prob = 0.4
l2_lambda = 0.001
init_lr = 0.001
decay_steps = 500
decay_rate = 0.99
clip_norm = 1
clipping = True
moving_avg_deacy = 0.999
num_gpus = 6

In [3]:
def average_gradients(tower_grads):
    average_grads = []
    for grad_and_vars in zip(*tower_grads):
        # Note that each grad_and_vars looks like the following:
        #   ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))
        grads = []
        for g, _ in grad_and_vars:
            # Add 0 dimension to the gradients to represent the tower.
            expanded_g = tf.expand_dims(g, 0)

            # Append on a 'tower' dimension which we will average over below.
            grads.append(expanded_g)

        # Average over the 'tower' dimension.
        grad = tf.concat(grads, 0)
        grad = tf.reduce_mean(grad, 0)

        # Keep in mind that the Variables are redundant because they are shared
        # across towers. So .. we will just return the first tower's pointer to
        # the Variable.
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads

In [4]:
# MODEL
device_num = 0
tower_grads = []
losses = []
predictions = []
predictions_pos = []

x = tf.placeholder('int32', [num_gpus, batch_size, max_sent_size], name="x")
y = tf.placeholder('int32', [num_gpus, batch_size, max_sent_size], name="y")
y_pos = tf.placeholder('int32', [num_gpus, batch_size, max_sent_size], name="y")
x_mask  = tf.placeholder('bool', [num_gpus, batch_size, max_sent_size], name='x_mask') 
sense_mask  = tf.placeholder('bool', [num_gpus, batch_size, max_sent_size], name='sense_mask')
is_train = tf.placeholder('bool', [], name='is_train')
word_emb_mat = tf.placeholder('float', [None, word_emb_size], name='emb_mat')
input_keep_prob = tf.cond(is_train,lambda:keep_prob, lambda:tf.constant(1.0))
pretrain = tf.placeholder('bool', [], name="pretrain")

global_step = tf.Variable(0, trainable=False, name="global_step")
learning_rate = tf.train.exponential_decay(init_lr, global_step, decay_steps, decay_rate, staircase=True)
summaries = []

def global_attention(input_x, input_mask, W_att):
    h_masked = tf.boolean_mask(input_x, input_mask)
    h_tanh = tf.tanh(h_masked)
    u = tf.matmul(h_tanh, W_att)
    a = tf.nn.softmax(u)
    c = tf.reduce_sum(tf.multiply(h_tanh, a), 0)  
    return c

with tf.variable_scope("word_embedding"):
    unk_word_emb_mat = tf.get_variable("word_emb_mat", dtype='float', shape=[unk_vocab_size, word_emb_size], initializer=tf.contrib.layers.xavier_initializer(uniform=True, seed=0, dtype=tf.float32))
    final_word_emb_mat = tf.concat([word_emb_mat, unk_word_emb_mat], 0)

with tf.variable_scope(tf.get_variable_scope()):
    for gpu_idx in range(num_gpus):
        if gpu_idx>int(num_gpus/2)-1:
            device_num = 1
        with tf.name_scope("model_{}".format(gpu_idx)) as scope, tf.device('/gpu:%d' % device_num):

            if gpu_idx > 0:
                    tf.get_variable_scope().reuse_variables()

            with tf.name_scope("word"):
                Wx = tf.nn.embedding_lookup(final_word_emb_mat, x[gpu_idx])  

            x_len = tf.reduce_sum(tf.cast(x_mask[gpu_idx], 'int32'), 1)

            with tf.variable_scope("lstm1"):
                cell_fw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
                cell_bw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

                d_cell_fw1 = tf.contrib.rnn.DropoutWrapper(cell_fw1, input_keep_prob=input_keep_prob)
                d_cell_bw1 = tf.contrib.rnn.DropoutWrapper(cell_bw1, input_keep_prob=input_keep_prob)

                (fw_h1, bw_h1), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw1, d_cell_bw1, Wx, sequence_length=x_len, dtype='float', scope='lstm1')
                h1 = tf.concat([fw_h1, bw_h1], 2)

            with tf.variable_scope("lstm2"):
                cell_fw2 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
                cell_bw2 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

                d_cell_fw2 = tf.contrib.rnn.DropoutWrapper(cell_fw2, input_keep_prob=input_keep_prob)
                d_cell_bw2 = tf.contrib.rnn.DropoutWrapper(cell_bw2, input_keep_prob=input_keep_prob)

                (fw_h2, bw_h2), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw2, d_cell_bw2, h1, sequence_length=x_len, dtype='float', scope='lstm2')
                h = tf.concat([fw_h2, bw_h2], 2)

            with tf.variable_scope("global_attention"):
                W_att = tf.get_variable("W_att", shape=[2*hidden_size, 1], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=gpu_idx*10))
                c = tf.expand_dims(global_attention(h[0], x_mask[gpu_idx][0], W_att), 0)
                for i in range(1, batch_size):
                    c = tf.concat([c, tf.expand_dims(global_attention(h[i], x_mask[gpu_idx][i], W_att), 0)], 0)
                cc = tf.expand_dims(c, 1)
                c_final = tf.tile(cc, [1, max_sent_size, 1])
    
            h_final = tf.concat([c_final, h], 2)
            flat_h_final = tf.reshape(h_final, [-1, tf.shape(h_final)[2]])
            
            with tf.variable_scope("hidden_layer"):
                W = tf.get_variable("W", shape=[4*hidden_size, 2*hidden_size], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=gpu_idx*20))
                b = tf.get_variable("b", shape=[2*hidden_size], initializer=tf.zeros_initializer())
                drop_flat_h_final = tf.nn.dropout(flat_h_final, input_keep_prob)
                flat_hl = tf.matmul(drop_flat_h_final, W) + b

            with tf.variable_scope("softmax_layer"):
                W = tf.get_variable("W", shape=[2*hidden_size, num_senses], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=gpu_idx*20))
                b = tf.get_variable("b", shape=[num_senses], initializer=tf.zeros_initializer())
                drop_flat_hl = tf.nn.dropout(flat_hl, input_keep_prob)
                flat_logits_sense = tf.matmul(drop_flat_hl, W) + b
                logits = tf.reshape(flat_logits_sense, [batch_size, max_sent_size, num_senses])
                predictions.append(tf.argmax(logits, 2))

            with tf.variable_scope("softmax_layer_pos"):
                W = tf.get_variable("W", shape=[4*hidden_size, num_pos], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=gpu_idx*30))
                b = tf.get_variable("b", shape=[num_pos], initializer=tf.zeros_initializer())
                drop_flat_h_pos = tf.nn.dropout(flat_h_final, input_keep_prob)
                flat_logits_pos = tf.matmul(drop_flat_h_pos, W) + b
                logits_pos = tf.reshape(flat_logits_pos, [batch_size, max_sent_size, num_pos])
                predictions_pos.append(tf.argmax(logits_pos, 2))


            float_sense_mask = tf.cast(sense_mask[gpu_idx], 'float')
            float_x_mask = tf.cast(x_mask[gpu_idx], 'float')

            loss = tf.contrib.seq2seq.sequence_loss(logits, y[gpu_idx], float_sense_mask, name="loss")
            loss_pos = tf.contrib.seq2seq.sequence_loss(logits_pos, y_pos[gpu_idx], float_x_mask, name="loss_")

            l2_loss = l2_lambda * tf.losses.get_regularization_loss()

            total_loss = tf.cond(pretrain, lambda:loss_pos, lambda:loss + loss_pos + l2_loss)

            summaries.append(tf.summary.scalar("loss_{}".format(gpu_idx), loss))
            summaries.append(tf.summary.scalar("loss_pos_{}".format(gpu_idx), loss_pos))
            summaries.append(tf.summary.scalar("total_loss_{}".format(gpu_idx), total_loss))


            optimizer = tf.train.AdamOptimizer(learning_rate)
            grads_vars = optimizer.compute_gradients(total_loss)

            clipped_grads = grads_vars
            if(clipping == True):
                clipped_grads = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in clipped_grads]

            tower_grads.append(clipped_grads)
            losses.append(total_loss)

tower_grads = average_gradients(tower_grads)
losses = tf.add_n(losses)/len(losses)
apply_grad_op = optimizer.apply_gradients(tower_grads, global_step=global_step)
summaries.append(tf.summary.scalar('total_loss', losses))
summaries.append(tf.summary.scalar('learning_rate', learning_rate))

for var in tf.trainable_variables():
    summaries.append(tf.summary.histogram(var.op.name, var))

variable_averages = tf.train.ExponentialMovingAverage(moving_avg_deacy, global_step)
variables_averages_op = variable_averages.apply(tf.trainable_variables())

train_op = tf.group(apply_grad_op, variables_averages_op)
saver = tf.train.Saver(tf.global_variables())
summary = tf.summary.merge(summaries)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2,3"
# print (device_lib.list_local_devices())
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())                          # For initializing all the variables
summary_writer = tf.summary.FileWriter(log_dir, sess.graph)          # For writing Summaries

In [6]:
save_period = 100
log_period = 100

def model(xx, yy, yy_pos, mask, smask, train_cond=True, pretrain_cond=False):
    num_batches = int(len(xx)/(batch_size*num_gpus))
    _losses = 0
    temp_loss = 0
    preds_sense = []
    true_sense = []
    preds_pos = []
    true_pos = []
    
    for j in range(num_batches): 
        
        s = j * batch_size * num_gpus
        e = (j+1) * batch_size * num_gpus
        xx_re = xx[s:e].reshape([num_gpus, batch_size, -1])
        yy_re = yy[s:e].reshape([num_gpus, batch_size, -1])
        yy_pos_re = yy_pos[s:e].reshape([num_gpus, batch_size, -1])
        mask_re = mask[s:e].reshape([num_gpus, batch_size, -1])
        smask_re = smask[s:e].reshape([num_gpus, batch_size, -1])
 
        feed_dict = {x:xx_re, y:yy_re, y_pos:yy_pos_re, x_mask:mask_re, sense_mask:smask_re, pretrain:pretrain_cond, is_train:train_cond, input_keep_prob:keep_prob, word_emb_mat:word_embedding}
        
        if(train_cond==True):
            _, _loss, step, _summary = sess.run([train_op, losses, global_step, summary], feed_dict)
            summary_writer.add_summary(_summary, step)
            
            temp_loss += _loss
            if((j+1)%log_period==0):
                print("Steps: {}".format(step), "Loss:{0:.4f}".format(temp_loss/log_period), ", Current Loss: {0:.4f}".format(_loss))
                temp_loss = 0
            if((j+1)%save_period==0):
                saver.save(sess, save_path=save_dir)                         
                
        else:
            _loss, pred, pred_pos = sess.run([total_loss, predictions, predictions_pos], feed_dict)
            for i in range(num_gpus):
                preds_sense.append(pred[i][smask_re[i]])
                true_sense.append(yy_re[i][smask_re[i]])
                preds_pos.append(pred_pos[i][mask_re[i]])
                true_pos.append(yy_pos_re[i][mask_re[i]])

        _losses +=_loss

    if(train_cond==False): 
        sense_preds = []
        sense_true = []
        pos_preds = []
        pos_true = []
        
        for preds in preds_sense:
            for ps in preds:      
                sense_preds.append(ps)  
        for trues in true_sense:
            for ts in trues:
                sense_true.append(ts)
        
        for preds in preds_pos:
            for ps in preds:      
                pos_preds.append(ps)      
        for trues in true_pos:
            for ts in trues:
                pos_true.append(ts)
                
        return _losses/num_batches, sense_preds, sense_true, pos_preds, pos_true

    return _losses/num_batches, step

def eval_score(yy, pred, yy_pos, pred_pos):
    f1 = f1_score(yy, pred, average='macro')
    accu = accuracy_score(yy, pred)
    f1_pos = f1_score(yy_pos, pred_pos, average='macro')
    accu_pos = accuracy_score(yy_pos, pred_pos)
    return f1*100, accu*100, f1_pos*100, accu_pos*100

def save_val_logs(temp):
    try:
        f = open(model_dir+ "/val_logs", "rb")
        old = pickle.load(f)
        f.close()
        f = open(model_dir+ "/val_logs", "wb")
        old.append(temp)
        pickle.dump(old, f)
    except FileNotFoundError:
        f = open(model_dir+ "/val_logs", "wb")
        pickle.dump([temp], f)
        f.close()

def save_train_logs(temp):
    try:
        f = open(model_dir+ "/train_logs", "rb")
        old = pickle.load(f)
        f.close()
        f = open(model_dir+ "/train_logs", "wb")
        old.append(temp)
        pickle.dump(old, f)
    except FileNotFoundError:
        f = open(model_dir+ "/train_logs", "wb")
        pickle.dump([temp], f)
        f.close()

In [7]:
x_id_train = train_data['x']
mask_train = train_data['x_mask']
sense_mask_train = train_data['sense_mask']
y_train = train_data['y']
y_pos_train = train_data['pos']

x_id_val = val_data['x']
mask_val = val_data['x_mask']
sense_mask_val = val_data['sense_mask']
y_val = val_data['y']
y_pos_val = val_data['pos']

In [ ]:
num_epochs = 1
val_period = 1
loss_collection = []
val_collection = []
pre_train_cond = True
for i in range(num_epochs):

    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    y_pos_train = y_pos_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, pretrain_cond=pre_train_cond)
    time_taken = time.time() - start_time
    save_train_logs([step, train_loss, time_taken])
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%val_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
        time_taken = time.time() - start_time
        save_val_logs([f1_, accu_, f1_pos_, accu_pos_, val_loss, time_taken])
        print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Steps: 100 Loss:5.3674 , Current Loss: 2.1611
Steps: 200 Loss:1.8464 , Current Loss: 1.0933
Steps: 300 Loss:0.7623 , Current Loss: 0.5904
Steps: 400 Loss:0.5191 , Current Loss: 0.4504
Steps: 500 Loss:0.4213 , Current Loss: 0.3914
Steps: 600 Loss:0.3600 , Current Loss: 0.3265
Steps: 700 Loss:0.3219 , Current Loss: 0.3012


In [ ]:
num_epochs = 10
val_period = 2
pre_train_cond = False
for i in range(num_epochs):

    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    y_pos_train = y_pos_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, pretrain_cond=pre_train_cond)
    time_taken = time.time() - start_time
    save_train_logs([step, train_loss, time_taken])
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%val_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
        time_taken = time.time() - start_time
        save_val_logs([f1_, accu_, f1_pos_, accu_pos_, val_loss, time_taken])
        print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Steps: 800 Loss:5.2592 , Current Loss: 3.2679
Steps: 900 Loss:3.0923 , Current Loss: 2.9597
Steps: 1000 Loss:2.7754 , Current Loss: 2.5682
Steps: 1100 Loss:2.5896 , Current Loss: 2.4611
Steps: 1200 Loss:2.4808 , Current Loss: 2.3712
Steps: 1300 Loss:2.4005 , Current Loss: 2.2374
Steps: 1400 Loss:2.3364 , Current Loss: 2.4100
Steps: 1500 Loss:2.2705 , Current Loss: 2.0756
Steps: 1600 Loss:2.2372 , Current Loss: 2.3246


In [ ]:
num_epochs = 10
val_period = 2
pre_train_cond = False
for i in range(num_epochs):

    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    y_pos_train = y_pos_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, pretrain_cond=pre_train_cond)
    time_taken = time.time() - start_time
    save_train_logs([step, train_loss, time_taken])
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%val_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
        time_taken = time.time() - start_time
        save_val_logs([f1_, accu_, f1_pos_, accu_pos_, val_loss, time_taken])
        print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Steps: 1700 Loss:2.1979 , Current Loss: 2.1717
Steps: 1800 Loss:2.1544 , Current Loss: 2.2598
Steps: 1900 Loss:2.1397 , Current Loss: 2.0750
Steps: 2000 Loss:2.1085 , Current Loss: 2.1254
Steps: 2100 Loss:2.0854 , Current Loss: 2.0996
Steps: 2200 Loss:2.0505 , Current Loss: 2.1495
Steps: 2300 Loss:2.0218 , Current Loss: 1.9948
Steps: 2400 Loss:2.0117 , Current Loss: 1.8594
Steps: 2500 Loss:1.9907 , Current Loss: 2.0386
Steps: 2600 Loss:1.9779 , Current Loss: 1.8969
Steps: 2700 Loss:1.9438 , Current Loss: 1.8520
Steps: 2800 Loss:1.9395 , Current Loss: 1.8981


In [ ]:
num_epochs = 10
val_period = 2
pre_train_cond = False
for i in range(num_epochs):

    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    y_pos_train = y_pos_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, pretrain_cond=pre_train_cond)
    time_taken = time.time() - start_time
    save_train_logs([step, train_loss, time_taken])
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%val_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
        time_taken = time.time() - start_time
        save_val_logs([f1_, accu_, f1_pos_, accu_pos_, val_loss, time_taken])
        print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Steps: 2900 Loss:1.9224 , Current Loss: 2.0388
Steps: 3000 Loss:1.9055 , Current Loss: 1.8295
Steps: 3100 Loss:1.9157 , Current Loss: 1.9636
Steps: 3200 Loss:1.8722 , Current Loss: 1.8092
Steps: 3300 Loss:1.8776 , Current Loss: 1.8689
Steps: 3400 Loss:1.8618 , Current Loss: 1.8058
Steps: 3500 Loss:1.8538 , Current Loss: 1.8207
Steps: 3600 Loss:1.8435 , Current Loss: 1.7502
Steps: 3700 Loss:1.8352 , Current Loss: 1.7308
Steps: 3800 Loss:1.8233 , Current Loss: 1.8277
Steps: 3900 Loss:1.8185 , Current Loss: 1.7640
Steps: 4000 Loss:1.8215 , Current Loss: 1.7741
Steps: 4100 Loss:1.8025 , Current Loss: 1.9120
Steps: 4200 Loss:1.7802 , Current Loss: 1.9101
Steps: 4300 Loss:1.7838 , Current Loss: 1.7946
Steps: 4400 Loss:1.7680 , Current Loss: 1.8407
Steps: 4500 Loss:1.7742 , Current Loss: 1.7113
Epoch: 1 , Step: 4571 , loss: 1.8357 , Time: 10830.0
Model Saved
Steps: 4671 Loss:1.7520 , Current Loss: 1.8807
Steps: 4771 Loss:1.7544 , Current Loss: 1.7509
Steps: 4871 Loss:1.7207 , Current Loss: 1.

In [ ]:
num_epochs = 10
val_period = 2
pre_train_cond = False
for i in range(num_epochs):

    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    y_pos_train = y_pos_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, pretrain_cond=pre_train_cond)
    time_taken = time.time() - start_time
    save_train_logs([step, train_loss, time_taken])
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%val_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
        time_taken = time.time() - start_time
        save_val_logs([f1_, accu_, f1_pos_, accu_pos_, val_loss, time_taken])
        print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Steps: 6071 Loss:1.6556 , Current Loss: 1.5446
Steps: 6171 Loss:1.6692 , Current Loss: 1.6034
Steps: 6271 Loss:1.6521 , Current Loss: 1.7231
Steps: 6371 Loss:1.6478 , Current Loss: 1.5646
Steps: 6471 Loss:1.6522 , Current Loss: 1.6472
Steps: 6571 Loss:1.6295 , Current Loss: 1.6840
Steps: 6671 Loss:1.6433 , Current Loss: 1.7914
Steps: 6771 Loss:1.6301 , Current Loss: 1.7140
Steps: 6871 Loss:1.6374 , Current Loss: 1.5301
Steps: 6971 Loss:1.6221 , Current Loss: 1.6431
Steps: 7071 Loss:1.6281 , Current Loss: 1.6017
Steps: 7171 Loss:1.6215 , Current Loss: 1.5612
Steps: 7271 Loss:1.6271 , Current Loss: 1.6493
Steps: 7371 Loss:1.5998 , Current Loss: 1.6637


In [ ]:
num_epochs = 10
val_period = 2
pre_train_cond = False
for i in range(num_epochs):

    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    y_pos_train = y_pos_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, pretrain_cond=pre_train_cond)
    time_taken = time.time() - start_time
    save_train_logs([step, train_loss, time_taken])
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%val_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
        time_taken = time.time() - start_time
        save_val_logs([f1_, accu_, f1_pos_, accu_pos_, val_loss, time_taken])
        print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Steps: 7471 Loss:1.6069 , Current Loss: 1.5844
Steps: 7571 Loss:1.6077 , Current Loss: 1.7320
Steps: 7671 Loss:1.5837 , Current Loss: 1.7231
Steps: 7771 Loss:1.6001 , Current Loss: 1.5729
Steps: 7871 Loss:1.5886 , Current Loss: 1.5351
Steps: 7971 Loss:1.5870 , Current Loss: 1.5669
Steps: 8071 Loss:1.5684 , Current Loss: 1.6432
Steps: 8171 Loss:1.5882 , Current Loss: 1.5069
Steps: 8271 Loss:1.5812 , Current Loss: 1.4797
Steps: 8371 Loss:1.5878 , Current Loss: 1.4984
Steps: 8471 Loss:1.5590 , Current Loss: 1.4710
Steps: 8571 Loss:1.5742 , Current Loss: 1.5925
Steps: 8671 Loss:1.5627 , Current Loss: 1.5128
Steps: 8771 Loss:1.5530 , Current Loss: 1.5961
Steps: 8871 Loss:1.5635 , Current Loss: 1.4247
Steps: 8971 Loss:1.5535 , Current Loss: 1.5449
Steps: 9071 Loss:1.5468 , Current Loss: 1.5294
Epoch: 1 , Step: 9142 , loss: 1.5763 , Time: 11145.0
Model Saved
Steps: 9242 Loss:1.5506 , Current Loss: 1.6275
Steps: 9342 Loss:1.5297 , Current Loss: 1.6551
Steps: 9442 Loss:1.5374 , Current Loss: 1.

In [9]:
num_epochs = 10
val_period = 2
pre_train_cond = False
for i in range(num_epochs):

    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    y_pos_train = y_pos_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, pretrain_cond=pre_train_cond)
    time_taken = time.time() - start_time
    save_train_logs([step, train_loss, time_taken])
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%val_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
        time_taken = time.time() - start_time
        save_val_logs([f1_, accu_, f1_pos_, accu_pos_, val_loss, time_taken])
        print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Steps: 14755 Loss:1.4351 , Current Loss: 1.3660
Steps: 14855 Loss:1.4388 , Current Loss: 1.5125
Steps: 14955 Loss:1.4286 , Current Loss: 1.4990
Steps: 15055 Loss:1.4329 , Current Loss: 1.3833
Steps: 15155 Loss:1.4495 , Current Loss: 1.5531
Steps: 15255 Loss:1.4287 , Current Loss: 1.5906
Steps: 15355 Loss:1.4412 , Current Loss: 1.3393
Steps: 15455 Loss:1.4202 , Current Loss: 1.4346
Steps: 15555 Loss:1.4190 , Current Loss: 1.4040
Steps: 15655 Loss:1.4353 , Current Loss: 1.3760
Steps: 15755 Loss:1.4289 , Current Loss: 1.3814
Steps: 15855 Loss:1.4229 , Current Loss: 1.3767
Steps: 15955 Loss:1.4179 , Current Loss: 1.4015
Steps: 16055 Loss:1.4180 , Current Loss: 1.5298
Steps: 16155 Loss:1.4209 , Current Loss: 1.2755
Steps: 16255 Loss:1.4120 , Current Loss: 1.4360
Steps: 16355 Loss:1.4028 , Current Loss: 1.3763
Epoch: 1 , Step: 16426 , loss: 1.4262 , Time: 11455.7
Model Saved
Steps: 16526 Loss:1.4044 , Current Loss: 1.3799
Steps: 16626 Loss:1.4269 , Current Loss: 1.3468
Steps: 16726 Loss:1.40

In [ ]:
num_epochs = 10
val_period = 2
pre_train_cond = False
for i in range(num_epochs):

    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    y_pos_train = y_pos_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, pretrain_cond=pre_train_cond)
    time_taken = time.time() - start_time
    save_train_logs([step, train_loss, time_taken])
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%val_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
        time_taken = time.time() - start_time
        save_val_logs([f1_, accu_, f1_pos_, accu_pos_, val_loss, time_taken])
        print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Steps: 32465 Loss:1.3049 , Current Loss: 1.2248
Steps: 32565 Loss:1.2873 , Current Loss: 1.2260
Steps: 32665 Loss:1.2704 , Current Loss: 1.2502
Steps: 32765 Loss:1.2931 , Current Loss: 1.2916
Steps: 32865 Loss:1.2917 , Current Loss: 1.2387
Steps: 32965 Loss:1.2802 , Current Loss: 1.3496
Steps: 33065 Loss:1.2910 , Current Loss: 1.2113
Steps: 33165 Loss:1.2964 , Current Loss: 1.3241
Steps: 33265 Loss:1.2837 , Current Loss: 1.4260
Steps: 33365 Loss:1.2833 , Current Loss: 1.2185
Steps: 33465 Loss:1.2998 , Current Loss: 1.1679
Steps: 33565 Loss:1.2888 , Current Loss: 1.2011
Steps: 33665 Loss:1.2870 , Current Loss: 1.3438
Steps: 33765 Loss:1.2976 , Current Loss: 1.3753
Steps: 33865 Loss:1.2916 , Current Loss: 1.2826
Steps: 33965 Loss:1.2803 , Current Loss: 1.1972
Steps: 34065 Loss:1.3049 , Current Loss: 1.3421
Epoch: 1 , Step: 34136 , loss: 1.2900 , Time: 10903.5
Model Saved
Steps: 34236 Loss:1.2772 , Current Loss: 1.3081
Steps: 34336 Loss:1.2645 , Current Loss: 1.3174


In [ ]:
num_epochs = 10
val_period = 2
pre_train_cond = False
for i in range(num_epochs):

    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    y_pos_train = y_pos_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, pretrain_cond=pre_train_cond)
    time_taken = time.time() - start_time
    save_train_logs([step, train_loss, time_taken])
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%val_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
        time_taken = time.time() - start_time
        save_val_logs([f1_, accu_, f1_pos_, accu_pos_, val_loss, time_taken])
        print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Steps: 34436 Loss:1.2826 , Current Loss: 1.3203
Steps: 34536 Loss:1.2909 , Current Loss: 1.2873
Steps: 34636 Loss:1.2842 , Current Loss: 1.2870
Steps: 34736 Loss:1.2721 , Current Loss: 1.2659
Steps: 34836 Loss:1.2901 , Current Loss: 1.3438
Steps: 34936 Loss:1.2827 , Current Loss: 1.2559
Steps: 35036 Loss:1.2842 , Current Loss: 1.3157
Steps: 35136 Loss:1.2874 , Current Loss: 1.3195
Steps: 35236 Loss:1.2878 , Current Loss: 1.2219
Steps: 35336 Loss:1.2830 , Current Loss: 1.2379
Steps: 35436 Loss:1.2766 , Current Loss: 1.2618
Steps: 35536 Loss:1.2784 , Current Loss: 1.3780
Steps: 35636 Loss:1.2676 , Current Loss: 1.1421
Steps: 35736 Loss:1.2676 , Current Loss: 1.3392
Steps: 35836 Loss:1.2822 , Current Loss: 1.2675
Steps: 35936 Loss:1.2747 , Current Loss: 1.2895
Steps: 36036 Loss:1.2805 , Current Loss: 1.2986
Epoch: 1 , Step: 36107 , loss: 1.2805 , Time: 10922.3
Model Saved
Steps: 36207 Loss:1.2716 , Current Loss: 1.2565
Steps: 36307 Loss:1.2795 , Current Loss: 1.2558
Steps: 36407 Loss:1.27

In [ ]:
num_epochs = 10
val_period = 2
pre_train_cond = False
for i in range(num_epochs):

    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    y_pos_train = y_pos_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, pretrain_cond=pre_train_cond)
    time_taken = time.time() - start_time
    save_train_logs([step, train_loss, time_taken])
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%val_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
        time_taken = time.time() - start_time
        save_val_logs([f1_, accu_, f1_pos_, accu_pos_, val_loss, time_taken])
        print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Steps: 45962 Loss:1.2408 , Current Loss: 1.2637
Steps: 46062 Loss:1.2183 , Current Loss: 1.0508
Steps: 46162 Loss:1.2250 , Current Loss: 1.3714
Steps: 46262 Loss:1.2178 , Current Loss: 1.3228
Steps: 46362 Loss:1.2224 , Current Loss: 1.2397
Steps: 46462 Loss:1.2096 , Current Loss: 1.0790
Steps: 46562 Loss:1.2142 , Current Loss: 1.1539
Steps: 46662 Loss:1.2289 , Current Loss: 1.0988
Steps: 46762 Loss:1.2253 , Current Loss: 1.0715
Steps: 46862 Loss:1.2199 , Current Loss: 1.1325
Steps: 46962 Loss:1.2121 , Current Loss: 1.1962
Steps: 47062 Loss:1.2146 , Current Loss: 1.0959
Steps: 47162 Loss:1.2176 , Current Loss: 1.0890
Steps: 47262 Loss:1.2186 , Current Loss: 1.1713


In [ ]:
num_epochs = 10
val_period = 2
pre_train_cond = False
for i in range(num_epochs):

    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    y_pos_train = y_pos_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, pretrain_cond=pre_train_cond)
    time_taken = time.time() - start_time
    save_train_logs([step, train_loss, time_taken])
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%val_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
        time_taken = time.time() - start_time
        save_val_logs([f1_, accu_, f1_pos_, accu_pos_, val_loss, time_taken])
        print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Steps: 47362 Loss:1.2001 , Current Loss: 1.1903
Steps: 47462 Loss:1.2224 , Current Loss: 1.1058
Steps: 47562 Loss:1.2119 , Current Loss: 1.2138
Steps: 47662 Loss:1.2174 , Current Loss: 1.2613
Steps: 47762 Loss:1.2113 , Current Loss: 1.1965
Steps: 47862 Loss:1.2086 , Current Loss: 1.0919
Steps: 47962 Loss:1.2051 , Current Loss: 1.2413
Steps: 48062 Loss:1.2059 , Current Loss: 1.2251
Steps: 48162 Loss:1.2126 , Current Loss: 1.1006
Steps: 48262 Loss:1.2056 , Current Loss: 1.2067
Steps: 48362 Loss:1.2115 , Current Loss: 1.1876
Steps: 48462 Loss:1.2202 , Current Loss: 1.1297
Steps: 48562 Loss:1.2222 , Current Loss: 1.2417
Steps: 48662 Loss:1.2038 , Current Loss: 1.1763
Steps: 48762 Loss:1.2122 , Current Loss: 1.1915
Steps: 48862 Loss:1.1997 , Current Loss: 1.2488
Steps: 48962 Loss:1.2198 , Current Loss: 1.2170
Epoch: 1 , Step: 49033 , loss: 1.2112 , Time: 10694.7
Model Saved
Steps: 49133 Loss:1.2012 , Current Loss: 1.1703
Steps: 49233 Loss:1.1981 , Current Loss: 1.2066
Steps: 49333 Loss:1.19

In [9]:
num_epochs = 10
val_period = 2
pre_train_cond = False
for i in range(num_epochs):

    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    y_pos_train = y_pos_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, pretrain_cond=pre_train_cond)
    time_taken = time.time() - start_time
    save_train_logs([step, train_loss, time_taken])
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%val_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
        time_taken = time.time() - start_time
        save_val_logs([f1_, accu_, f1_pos_, accu_pos_, val_loss, time_taken])
        print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Steps: 61830 Loss:1.1579 , Current Loss: 1.0321
Steps: 61930 Loss:1.1521 , Current Loss: 1.1229
Steps: 62030 Loss:1.1666 , Current Loss: 1.2002
Steps: 62130 Loss:1.1534 , Current Loss: 1.1285
Steps: 62230 Loss:1.1583 , Current Loss: 1.0084
Steps: 62330 Loss:1.1593 , Current Loss: 1.0921
Steps: 62430 Loss:1.1541 , Current Loss: 1.1498
Steps: 62530 Loss:1.1484 , Current Loss: 1.1735
Steps: 62630 Loss:1.1605 , Current Loss: 1.1531
Steps: 62730 Loss:1.1446 , Current Loss: 1.1768
Steps: 62830 Loss:1.1490 , Current Loss: 1.2653
Steps: 62930 Loss:1.1532 , Current Loss: 1.1785
Steps: 63030 Loss:1.1510 , Current Loss: 1.0946
Steps: 63130 Loss:1.1483 , Current Loss: 1.1987
Steps: 63230 Loss:1.1545 , Current Loss: 1.0606
Steps: 63330 Loss:1.1536 , Current Loss: 1.2039
Steps: 63430 Loss:1.1545 , Current Loss: 1.1147
Epoch: 1 , Step: 63501 , loss: 1.1533 , Time: 10642.6
Model Saved
Steps: 63601 Loss:1.1432 , Current Loss: 1.1006
Steps: 63701 Loss:1.1529 , Current Loss: 1.2350
Steps: 63801 Loss:1.14

In [ ]:
num_epochs = 10
val_period = 2
pre_train_cond = False
for i in range(num_epochs):

    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    y_pos_train = y_pos_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, pretrain_cond=pre_train_cond)
    time_taken = time.time() - start_time
    save_train_logs([step, train_loss, time_taken])
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%val_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
        time_taken = time.time() - start_time
        save_val_logs([f1_, accu_, f1_pos_, accu_pos_, val_loss, time_taken])
        print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

Steps: 79540 Loss:1.1178 , Current Loss: 1.2695
Steps: 79640 Loss:1.1092 , Current Loss: 1.0448
Steps: 79740 Loss:1.1106 , Current Loss: 1.1329
Steps: 79840 Loss:1.1163 , Current Loss: 1.0621
Steps: 79940 Loss:1.1007 , Current Loss: 1.0109
Steps: 80040 Loss:1.1170 , Current Loss: 1.0699
Steps: 80140 Loss:1.1124 , Current Loss: 1.0198
Steps: 80240 Loss:1.1125 , Current Loss: 1.0548
Steps: 80340 Loss:1.0992 , Current Loss: 1.1341
Steps: 80440 Loss:1.1191 , Current Loss: 1.0745
Steps: 80540 Loss:1.1051 , Current Loss: 1.0424
Steps: 80640 Loss:1.1212 , Current Loss: 1.3158
Steps: 80740 Loss:1.1037 , Current Loss: 1.1604
Steps: 80840 Loss:1.1198 , Current Loss: 1.2913
Steps: 80940 Loss:1.1110 , Current Loss: 1.1918
Steps: 81040 Loss:1.1207 , Current Loss: 1.0651
Steps: 81140 Loss:1.1172 , Current Loss: 1.1293
Epoch: 1 , Step: 81211 , loss: 1.1127 , Time: 10584.8
Model Saved
Steps: 81311 Loss:1.1207 , Current Loss: 1.0546
Steps: 81411 Loss:1.1127 , Current Loss: 1.0906
Steps: 81511 Loss:1.11

In [ ]:
num_epochs = 10
val_period = 2
pre_train_cond = False
for i in range(num_epochs):

    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    sense_mask_train = sense_mask_train[random]
    y_pos_train = y_pos_train[random]
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, pretrain_cond=pre_train_cond)
    time_taken = time.time() - start_time
    save_train_logs([step, train_loss, time_taken])
    print("Epoch: {}".format(i+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
    saver.save(sess, save_path=save_dir)                         
    print("Model Saved")
    
    if((i+1)%val_period==0):
        start_time = time.time()
        val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
        time_taken = time.time() - start_time
        save_val_logs([f1_, accu_, f1_pos_, accu_pos_, val_loss, time_taken])
        print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))

In [ ]:
start_time = time.time()
train_loss, train_pred, train_true, train_pred_pos, train_true_pos = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, train_cond=False)        
f1_, accu_, f1_pos_, accu_pos_ = etrain_score(train_true, train_pred, train_true_pos, train_pred_pos)
time_taken = time.time() - start_time
print("train: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))

In [ ]:
saver.restore(sess, save_dir)